## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [ ]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [1]:
import os
import re
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter, defaultdict

In [2]:
import warnings
warnings.filterwarnings('ignore')

**1. получить коллекцию документов**

In [3]:
main_dir = '/Users/hlibkozliuk/Downloads/Friends'
files_list = []

### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов

In [4]:
etitles = []

In [5]:
for root, dirs, files in os.walk(main_dir):
    etitles += files
    for name in files:
        if name != '.DS_Store':
            files_list.append(os.path.join(root, name))

**2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку**

In [7]:
def clean_data(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[\s]{2,}', ' ', text)

    return text

In [8]:
morph = MorphAnalyzer()

In [9]:
def normalize(text):
    text = clean_data(text)

    tokens = word_tokenize(text)
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    
    return ' '.join(lemmas)

In [10]:
res_texts = []

In [11]:
# nltk.download('punkt')

In [12]:
for file in files_list:
    with open(file, 'r', encoding='utf-8') as f:
        f = f.read()
    
    normalized = normalize(f)
    res_texts.append(normalized)

**3. получить матрицу терм-документ, написать функцию поиска по ней**

In [13]:
### постройте матрицу терм-документ

count_vect = CountVectorizer()
X = count_vect.fit_transform(res_texts)
term_doc_matrix = pd.DataFrame(X.toarray(), index=etitles[1:], columns=count_vect.get_feature_names())

In [56]:
term_doc_matrix = term_doc_matrix.transpose()
term_doc_matrix.head()

,after,again,ahh,all,and,are,au,aнгел,bay,behind,...,ящичек,ёй,ёкнуть,ёлка,ёлочный,ёпэрэсотэ,ёрл,ёрш,ёршик,ёще
Friends - 2x08 - The One With The List.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 2x24 - The One With Barry And Mindy's Wedding.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 2x16 - The One Where Joey Moves Out.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 2x07 - The One Where Ross Finds Out.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
### напишите функцию булева поиска по построенной матрице
def boolean_search() -> list:
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    """
    return


#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джои & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)'
]

**4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
значение - список файлов, в которых это слово встречается**

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [53]:
words = count_vect.get_feature_names()

In [60]:
def inverted_index() -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    
    inv_idx = defaultdict(list)
    
    for word in words:
        values = term_doc_matrix[term_doc_matrix[word] > 0]
        for val in values.transpose():
            inv_idx[word].append(val)
            
    return inv_idx

idx = inverted_index()

In [68]:
for key, value in idx.items():
      print(key, "=", value)

after = ['Friends - 7x09 - The One With All The Candy.ru.txt']
again = ["Friends - 5x19 - The One Where Ross Can't Flirt.ru.txt"]
ahh = ['Friends - 5x02 - The One With All The Kissing.ru.txt']
all = ['Friends - 5x18 - The One Where Rachel Smokes.ru.txt', 'Friends - 6x01 - The One After Vegas.ru.txt']
and = ['Friends - 4x01 - The One With The Jellyfish.Tv.ru.txt', "Friends - 6x19 - The One With Joey's Fridge.ru.txt"]
are = ['Friends - 4x01 - The One With The Jellyfish.Tv.ru.txt']
au = ["Friends - 5x19 - The One Where Ross Can't Flirt.ru.txt"]
aнгел = ['Friends - 7x13 - The One Where Rosita Dies.ru.txt']
bay = ["Friends - 7x02 - The One With Rachel's Book.ru.txt"]
behind = ['Friends - 6x12 - The One With The Joke.ru.txt']
bodingtons = ['Friends - 5x02 - The One With All The Kissing.ru.txt']
bodingtonsдааа = ['Friends - 5x02 - The One With All The Kissing.ru.txt']
bonjour = ["Friends - 7x01 - The One With Monica's Thunder.ru.txt"]
by = ['Friends - 2x03 - The One Where Heckles Dies.ru.txt'

бигуди = ["Friends - 2x01 - The One With Ross's New Girlfriend.DVDRip.ru.txt"]
биение = ['Friends - 4x17 - The One With The Free Porn.ru.txt']
бижан = ['Friends - 2x02 - The One With The Breast Milk.ru.txt']
бижутерия = ["Friends - 5x19 - The One Where Ross Can't Flirt.ru.txt", "Friends - 3x14 - The One With Phoebe's Ex-Partner.ru.txt"]
бизнес = ['Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt', 'Friends - 5x24-25 - The One In Vegas (2).ru.txt', "Friends - 4x13 - The One With Rachel's Crush.ru.txt", "Friends - 4x09 - The One Where They're Going To Party!.ru.txt", 'Friends - 6x25-26 - The One With The Proposal (2).ru.txt', 'Friends - 1x17 - The One With Two Parts (2).ru.txt']
бикини = ['Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt', 'Friends - 3x15 - The One Where Ross And Rachel Take A Break (1).ru.txt', 'Friends - 3x16 - The One With The Morning After (2).ru.txt', 'Friends - 3x01 - The One With The Princess Leia Fantasy.ru.txt', "Friends - 7x22 - The On

взросление = ['Friends - 2x13 - The One After The Super Bowl (2).ru.txt']
взрослеть = ['Friends - 2x17 - The One Where Eddie Moves In.ru.txt', "Friends - 4x18 - The One With Rachel's New Dress.ru.txt", "Friends - 3x11 - The One Where Chandler Can't Remember Which Sister.ru.txt"]
взрослый = ["Friends - 2x24 - The One With Barry And Mindy's Wedding.ru.txt", 'Friends - 2x17 - The One Where Eddie Moves In.ru.txt', 'Friends - 2x15 - The One Where Ross And Rachel...You Know.ru.txt', 'Friends - 2x21 - The One With The Bullies.ru.txt', 'Friends - 2x02 - The One With The Breast Milk.ru.txt', 'Friends - 2x12 - The One After The Super Bowl (1).ru.txt', 'Friends - 5x05 - The One With The Kips.ru.txt', "Friends - 5x09 - The One With Ross's Sandwich.ru.txt", 'Friends - 5x06 - The One With The Yeti.ru.txt', 'Friends - 5x11 - The One With All The Resolutions.ru.txt', 'Friends - 5x03 - The One Hundredth.ru.txt', "Friends - 4x11 - The One With Phoebe's Uterus.ru.txt", "Friends - 4x05 - The One With Joey

вырывать = ["Friends - 4x23-24 - The One With Ross's Wedding (1).ru.txt", 'Friends - 3x20 - The One With The Dollhouse.ru.txt']
вырядиться = ['Friends - 2x15 - The One Where Ross And Rachel...You Know.ru.txt', 'Friends - 5x04 - The One Where Phoebe Hates PBS.ru.txt', "Friends - 5x12 - The One With Chandler's Work Laugh.ru.txt", "Friends - 6x05 - The One With Joey's Porsche.ru.txt", "Friends - 7x22 - The One With Chandler's Dad.ru.txt"]
высадить = ['Friends - 5x01 - The One After Ross Says Rachel.ru.txt']
высадиться = ['Friends - 4x10 - The One With The Girl From Poughkeepsie.ru.txt']
высасывать = ['Friends - 5x20 - The One With The Ride Along.ru.txt', 'Friends - 7x17 - The One With The Cheap Wedding Dress.ru.txt']
выселение = ["Friends - 2x19 - The One Where Eddie Won't Go.ru.txt", 'Friends - 5x08 - The One With The Thanksgiving Flashbacks.ru.txt']
выселить = ["Friends - 5x09 - The One With Ross's Sandwich.ru.txt", 'Friends - 4x04 - The One With The Ballroom Dancing.ru.txt']
высидеть =

дейвид = ['Friends - 4x19 - The One With All The Haste.ru.txt', 'Friends - 4x15 - The One With All The Rugby.ru.txt']
дейвис = ['Friends - 3x03 - The One With The Jam.ru.txt']
дейльюис = ['Friends - 3x05 - The One With Frank Jr..ru.txt']
дейсвительный = ["Friends - 2x04 - The One With Phoebe's Husband.ru.txt"]
действительно = ['Friends - 2x07 - The One Where Ross Finds Out.ru.txt', 'Friends - 2x17 - The One Where Eddie Moves In.ru.txt', "Friends - 2x04 - The One With Phoebe's Husband.ru.txt", 'Friends - 2x21 - The One With The Bullies.ru.txt', "Friends - 2x09 - The One With Phoebe's Dad.ru.txt", "Friends - 2x19 - The One Where Eddie Won't Go.ru.txt", 'Friends - 2x23 - The One With The Chicken Pox.ru.txt', 'Friends - 2x05 - The One With Five Steaks And An Eggplant.ru.txt', "Friends - 5x09 - The One With Ross's Sandwich.ru.txt", 'Friends - 5x03 - The One Hundredth.ru.txt', 'Friends - 5x02 - The One With All The Kissing.ru.txt', 'Friends - 5x21 - The One With The Ball.ru.txt', "Friends - 

...

 = ['Friends - 2x23 - The One With The Chicken Pox.ru.txt']
статистика = ['Friends - 2x23 - The One With The Chicken Pox.ru.txt', 'Friends - 1x15 - The One With The Stoned Guy.ru.txt']
статистикстажер = ['Friends - 2x23 - The One With The Chicken Pox.ru.txt']
статистический = ['Friends - 1x15 - The One With The Stoned Guy.ru.txt']
статический = ['Friends - 3x13 - The One Where Monica And Richard Are Just Friends.ru.txt']
статус = ['Friends - 1x15 - The One With The Stoned Guy.ru.txt']
статуя = ['Friends - 2x11 - The One With The Lesbian Wedding.ru.txt', 'Friends - 3x01 - The One With The Princess Leia Fantasy.ru.txt', 'Friends - 7x05 - The One With The Engagement Picture.ru.txt', 'Friends - 7x11 - The One With All The Cheesecakes.ru.txt']
стать = ['Friends - 2x08 - The One With The List.ru.txt', 'Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt', "Friends - 2x04 - The One With Phoebe's Husband.ru.txt", 'Friends - 2x15 - The One Where Ross And Rachel...You Know.ru.txt', "Fr

тон = ["Friends - 4x05 - The One With Joey's New Girlfriend.ru.txt", 'Friends - 1x15 - The One With The Stoned Guy.ru.txt', 'Friends - 1x09 - The One Where Underdog Gets Away.ru.txt', "Friends - 7x08 - The One Where Chandler Doesn't Like Dogs.ru.txt"]
тоненький = ['Friends - 6x10 - The One With The Routine.ru.txt', 'Friends - 7x16 - The One With The Truth About London.ru.txt']
тонер = ['Friends - 7x13 - The One Where Rosita Dies.ru.txt']
тони = ["Friends - 2x24 - The One With Barry And Mindy's Wedding.ru.txt", 'Friends - 2x17 - The One Where Eddie Moves In.ru.txt', 'Friends - 4x07 - The One Where Chandler Crosses The Line.ru.txt', 'Friends - 4x06 - The One With The Dirty Girl.ru.txt', "Friends - 4x09 - The One Where They're Going To Party!.ru.txt", 'Friends - 3x01 - The One With The Princess Leia Fantasy.ru.txt', 'Friends - 6x17 - The One With Unagi.ru.txt', 'Friends - 1x15 - The One With The Stoned Guy.ru.txt', 'Friends - 1x18 - The One With All The Poker.ru.txt', 'Friends - 1x05 - Th

фанат = ['Friends - 2x06 - The One With The Baby On The Bus.ru.txt', 'Friends - 2x12 - The One After The Super Bowl (1).ru.txt', "Friends - 7x22 - The One With Chandler's Dad.ru.txt", "Friends - 7x15 - The One With Joey's New Brain.ru.txt", "Friends - 7x07 - The One With Ross's Library Book.ru.txt"]
фанатка = ['Friends - 2x12 - The One After The Super Bowl (1).ru.txt', "Friends - 5x19 - The One Where Ross Can't Flirt.ru.txt", "Friends - 7x15 - The One With Joey's New Brain.ru.txt"]
фанаткать = ['Friends - 2x12 - The One After The Super Bowl (1).ru.txt', 'Friends - 6x15 - The One That Could Have Been (1).ru.txt']
фановый = ["Friends - 7x15 - The One With Joey's New Brain.ru.txt"]
фаном = ['Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt']
фантазия = ['Friends - 4x17 - The One With The Free Porn.ru.txt', 'Friends - 3x01 - The One With The Princess Leia Fantasy.ru.txt', 'Friends - 1x06 - The One With The Butt.ru.txt']
фантастика = ["Friends - 4x13 - The One With Rachel's Cru

чек = ['Friends - 2x08 - The One With The List.ru.txt', 'Friends - 2x02 - The One With The Breast Milk.ru.txt', 'Friends - 2x14 - The One With The Prom Video.ru.txt', 'Friends - 5x15 - The One With The Girl Who Hits Joey.ru.txt', 'Friends - 3x07 - The One With The Race Car Bed.ru.txt', 'Friends - 3x08 - The One With The Giant Poking Device.ru.txt', 'Friends - 3x18 - The One With The Hypnosis Tape.ru.txt', 'Friends - 3x05 - The One With Frank Jr..ru.txt', 'Friends - 6x04 - The One Where Joey Loses His Insurance.ru.txt', 'Friends - 6x12 - The One With The Joke.ru.txt', "Friends - 6x14 - The One Where Chandler Can't Cry (2).ru.txt", 'Friends - 6x23 - The One With The Ring.ru.txt', "Friends - 6x19 - The One With Joey's Fridge.ru.txt", 'Friends - 1x17 - The One With Two Parts (2).ru.txt', 'Friends - 1x04 - The One With George Stephanopoulos.ru.txt', "Friends - 7x07 - The One With Ross's Library Book.ru.txt"]
чековый = ["Friends - 4x09 - The One Where They're Going To Party!.ru.txt", 'Friend

янки = ['Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt']
янкиз = ["Friends - 7x20 - The One With Rachel's Big Kiss.ru.txt"]
янковский = ['Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt']
японец = ['Friends - 6x17 - The One With Unagi.ru.txt']
япония = ['Friends - 3x21 - The One With A Chick And A Duck.ru.txt']
японский = ['Friends - 6x12 - The One With The Joke.ru.txt', 'Friends - 1x08 - The One Where Nana Dies Twice.ru.txt']
япросить = ['Friends - 5x02 - The One With All The Kissing.ru.txt']
яркий = ['Friends - 2x15 - The One Where Ross And Rachel...You Know.ru.txt', 'Friends - 5x23-24 - The One In Vegas (1).ru.txt', 'Friends - 3x21 - The One With A Chick And A Duck.ru.txt', "Friends - 6x08 - The One With Ross's Teeth.ru.txt", 'Friends - 7x06 - The One With The Nap Partners.ru.txt', 'Friends - 7x05 - The One With The Engagement Picture.ru.txt']
ярко = ['Friends - 1x10 - The One With The Monkey.ru.txt', 'Friends - 1x09 - The One Where Underdog Gets Away.r

**5. вывести кусочек индекса в виде таблицы**

In [76]:
inv_idx_df = pd.DataFrame()
inv_idx_df['Word'] = idx.keys()
inv_idx_df['Episode(s)'] = idx.values()

In [77]:
inv_idx_df.head()

,Word,Episode(s)
0,after,[Friends - 7x09 - The One With All The Candy.r...
1,again,[Friends - 5x19 - The One Where Ross Can't Fli...
2,ahh,[Friends - 5x02 - The One With All The Kissing...
3,all,[Friends - 5x18 - The One Where Rachel Smokes....
4,and,[Friends - 4x01 - The One With The Jellyfish.T...


С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


In [119]:
frequency = list(term_doc_matrix.sum(axis=0))

In [120]:
inv_idx_df['Word frequency'] = frequency

In [121]:
inv_idx_df.head()

,Word,Episode(s),Word frequency
0,after,[Friends - 7x09 - The One With All The Candy.r...,1
1,again,[Friends - 5x19 - The One Where Ross Can't Fli...,1
2,ahh,[Friends - 5x02 - The One With All The Kissing...,1
3,all,[Friends - 5x18 - The One Where Rachel Smokes....,3
4,and,[Friends - 4x01 - The One With The Jellyfish.T...,3


**Какое слово является самым частотным?**

In [122]:
inv_idx_df.sort_values('Word frequency', ascending=False).head()

,Word,Episode(s),Word frequency
13647,ты,[Friends - 2x08 - The One With The List.ru.txt...,11177
6862,не,[Friends - 2x08 - The One With The List.ru.txt...,9439
14878,что,[Friends - 2x08 - The One With The List.ru.txt...,8753
15335,это,[Friends - 2x08 - The One With The List.ru.txt...,7506
1127,быть,[Friends - 2x08 - The One With The List.ru.txt...,4763


**_ты_** является самым частотным словом.

**Какое самым редким?**

In [129]:
inv_idx_df.loc[inv_idx_df['Word frequency'] == 1].head(10)

,Word,Episode(s),Word frequency
0,after,[Friends - 7x09 - The One With All The Candy.r...,1
1,again,[Friends - 5x19 - The One Where Ross Can't Fli...,1
2,ahh,[Friends - 5x02 - The One With All The Kissing...,1
5,are,[Friends - 4x01 - The One With The Jellyfish.T...,1
6,au,[Friends - 5x19 - The One Where Ross Can't Fli...,1
7,aнгел,[Friends - 7x13 - The One Where Rosita Dies.ru...,1
8,bay,[Friends - 7x02 - The One With Rachel's Book.r...,1
9,behind,[Friends - 6x12 - The One With The Joke.ru.txt],1
11,bodingtonsдааа,[Friends - 5x02 - The One With All The Kissing...,1
12,bonjour,[Friends - 7x01 - The One With Monica's Thunde...,1


**Какой набор слов есть во всех документах коллекции?**

In [140]:
common_words = []
for word in idx:
    if len(idx[word]) == 165:
        common_words.append(word)

In [142]:
common_words

['быть',
 'весь',
 'да',
 'думать',
 'если',
 'ещё',
 'знать',
 'как',
 'мой',
 'мочь',
 'мы',
 'на',
 'не',
 'нет',
 'но',
 'ну',
 'он',
 'она',
 'просто',
 'так',
 'такой',
 'тот',
 'ты',
 'хотеть',
 'что',
 'это',
 'этот']

**Какой сезон был самым популярным у Чендлера? у Моники?**

## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [ ]:
### реализуйте эту функцию ранжирования 
from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return 

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [ ]:
def compute_sim() -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return 


def get_search_result() -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    return 